# Tavily research endpoint with polling

Research calls are handled by an agent that may take several minutes to complete. To accommodate this, we've provided an endpoint that allows you to poll for the research results as they become available.

In [ ]:
%pip install -qU tavily-python

In [10]:
import getpass
import os
from tavily import TavilyClient
import time

from IPython.display import display, Markdown

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY:\n")

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

## Make a Polling Request

Let's begin by sending a request to the research API

In [11]:
result = tavily_client.research(input="What are the latest developments in artificial intelligence?", model="mini") # model = "mini" | "pro" | "auto"

After submitting a research request to the endpoint, you'll receive a response that includes a "request_id".


In [12]:
request_id = result["request_id"]

After submitting your research request, you can track its progress using the `get_research` method along with the received request_id.

The response will include a "status" field that reflects the current stage of your request. The possible status values are:

- "pending": Your request is waiting to be processed.
- "in progress": Your request is currently being processed.
- "completed": The research has finished and results are available.
- "failed": There was an issue processing your request.


In [13]:
response = tavily_client.get_research(request_id)

When the status is `completed` you can access the report from the "content" field.

In [ ]:
# Poll until the research is completed
while response["status"] not in ["completed", "failed"]:
    print(f"Status: {response['status']}... polling again in 10 seconds")
    time.sleep(10)
    response = tavily_client.get_research(request_id)

# Check if the research completed successfully
if response["status"] == "failed":
    raise RuntimeError(f"Research failed: {response.get('error', 'Unknown error')}")

print("Research Complete!")
report = Markdown(response["content"])
display(report)

In [ ]:
response['sources']